In [1]:
!pip install ultralytics opencv-python pillow

In [11]:
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
from PIL import Image

# Hide the root window
root = tk.Tk()
root.withdraw()

# Open a file dialog to select an image file
image_path = filedialog.askopenfilename(
    title='Select an image file',
    filetypes=[('Image files', '*.jpg *.jpeg *.png *.bmp *.tiff')]
)

if not image_path:
    raise ValueError("No file selected.")

# 📷 Load and preprocess with OpenCV
image_bgr = cv2.imread(image_path)

# Resize large images to max 1024px width/height
max_dim = 1024
h, w = image_bgr.shape[:2]
if max(h, w) > max_dim:
    scale = max_dim / max(h, w)
    image_bgr = cv2.resize(image_bgr, (int(w * scale), int(h * scale)))

# Convert BGR to RGB (YOLO expects RGB)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)


In [12]:
# # 🤖 STEP 2: Run YOLOv8 on COCO
# from ultralytics import YOLO

# model = YOLO("yolov8n.pt")  # You can use yolov8s.pt for better accuracy

# results = model(image_rgb)

# # Get first valid detection from allowed classes
# allowed_classes = ['person', 'cat', 'dog']
# subject_detected = None

# for box in results[0].boxes:
#     cls = int(box.cls[0])
#     label = model.names[cls]
#     if label in allowed_classes:
#         subject_detected = label
#         break

# if subject_detected:
#     print(f"✅ Detected subject: {subject_detected}")
# else:
#     raise ValueError("❌ No valid subject (person/cat/dog) detected!")
# 🤖 Run YOLOv8 on COCO
from ultralytics import YOLO

# Load YOLO model
model = YOLO("yolov8n.pt")  # You can upgrade to yolov8s.pt for better results

# Run detection on image
results = model(image_rgb)

# Get list of all detected class names
detected_classes = []
for box in results[0].boxes:
    cls_id = int(box.cls[0])
    label = model.names[cls_id]
    if label not in detected_classes:
        detected_classes.append(label)

# Print the results
if detected_classes:
    print("✅ Detected objects:", ", ".join(detected_classes))
else:
    raise ValueError("❌ No objects detected by YOLOv8!")





0: 448x640 2 persons, 51.5ms
Speed: 1.7ms preprocess, 51.5ms inference, 4.7ms postprocess per image at shape (1, 3, 448, 640)
✅ Detected objects: person


In [27]:

# 🧠 Build the prompt using all detected object names
object_list = ", ".join(detected_classes)
# 🎭 Choose your style
theme_prompts = {
    "caricature": (
        "A vivid caricature of the input image,  color palette. Focus on enhancing and transforming the colors for artistic effect, but keep the original shapes, proportions, and structure unchanged. Do not distort or exaggerate the physical features—let the color treatment be the main artistic element. Digital caricature art, highly detailed, professional finish."
    ),
    "oil_painting": (
        "An oil painting version of the input image, rich textures, visible brushstrokes, classic fine art style, deep colors, realistic lighting, elegant composition, masterpiece, highly detailed.I want the structure of the image to stay the same , dont displace the component of the image . The components are :{object_list}"
    ),
    "cartoon": (
        "A vibrant cartoon art style of the input image, clean lines, bold colors, simplified shapes, expressive cartoon features, digital illustration, professional cartoon art style, highly detailed.I want the structure of the image to stay the same , dont displace the component of the image . The components list is  :{object_list}"
    ),
    "cinematic": (
        "A cinematic version of the input image, dramatic lighting, filmic color grading, shallow depth of field, widescreen aspect ratio, high contrast, atmospheric, movie still, highly detailed.I want the structure of the image to stay the same , dont displace the component of the image . The components are :{object_list}"
    ),
    "shadowed": (
        "A shadowed version of the input image, strong shadows, moody atmosphere, dramatic contrast, chiaroscuro lighting, mysterious, deep blacks, artistic composition, highly detailed.I want the structure of the image to stay the same , dont displace the component of the image . The components are :{object_list}"
    ),
    "horror": (
        "A horror-themed art style of the input image, dark and eerie atmosphere, unsettling details, creepy lighting, ominous shadows, horror movie art style, intense mood, highly detailed, blood everywhere .I want the structure of the image to stay the same , dont displace the component of the image . The components are :{object_list}"
    ),
    "van_gogh": (
        "A Van Gogh style painting of the input image, expressive swirling brushstrokes, thick impasto texture, vibrant and dynamic colors, inspired by Vincent van Gogh’s masterpieces, highly detailed, starry night mood. .I want the structure of the image to stay the same , dont displace the component of the image . The components are :{object_list}"
    ),
    "cubism": (
        "A cubist interpretation of the input image, fragmented and geometric shapes, multiple perspectives, abstract forms, muted earth tones and bold color blocks, inspired by Pablo Picasso and Georges Braque, dont degrade colors modern art, highly detailed.I want the structure of the image to stay the same , dont displace the component of the image . The components are :{object_list}"    ),
    "pop_art": (
        "A vibrant pop art version of the input image, different pop art colors  comic book style, inspired by Andy Warhol and Roy Lichtenstein, playful and graphic, highly detailed, modern pop culture aesthetic. dont displace the component of the image . The components are :{object_list}"    )
}

selected_theme = "van_gogh"

prompt = theme_prompts[selected_theme].format(object_list=object_list)



print("📜 Prompt for stylization:", prompt)

📜 Prompt for stylization: A Van Gogh style painting of the input image, expressive swirling brushstrokes, thick impasto texture, vibrant and dynamic colors, inspired by Vincent van Gogh’s masterpieces, highly detailed, starry night mood. .I want the structure of the image to stay the same , dont displace the component of the image . The components are :person


In [5]:
# 📦 Install required Hugging Face libraries
!pip install diffusers transformers accelerate safetensors

In [4]:
#using SDXL
from diffusers import StableDiffusionXLImg2ImgPipeline
import torch
from PIL import Image
import numpy as np

# ✅ Load the SDXL img2img pipeline (first run may take a few minutes)
pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
            # Remove this line for CPU or if you get errors
    safety_checker=None
).to("mps")
pipe.enable_attention_slicing()

Keyword arguments {'safety_checker': None} are not expected by StableDiffusionXLImg2ImgPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [28]:
#Convert and resize input image to SDXL's default resolution (1024x1024)
image_pil = Image.fromarray(image_rgb).convert("RGB")
image_pil = image_pil.resize((1024, 1024))

# Run SDXL stylization
output = pipe(
    prompt=prompt,
    image=image_pil,
    strength=0.6,         # Controls how much the prompt changes the original
    guidance_scale=10    # Controls prompt adherence; 7.5–14 is typical
    # generator=generator  # Uncomment if you want reproducibility
)

# Show and save the result 431865++
stylized_image = output.images[0]
stylized_image.save("final_output_sdxl.png")
stylized_image.show

  0%|          | 0/30 [00:00<?, ?it/s]

<bound method Image.show of <PIL.Image.Image image mode=RGB size=1024x1024 at 0x346BCD9F0>>